In [ ]:
from flask import Flask,render_template,request,redirect,url_for,session
from flask_pymongo import PyMongo
from werkzeug.utils import secure_filename
import os

from flask import flash

# from decorators import login_required
from functools import wraps

app = Flask(__name__)
app.config['SECRET_KEY'] = os.urandom(24)
app.config["MONGO_URI"] = "mongodb://localhost:27017/recommend"
mongo = PyMongo(app)

def login_required(func):
    @wraps(func)
    def wrapper(*args,**kwargs):
        print('session get account: ',session.get('account'))
        if session.get('account'):
            return func(*args,**kwargs)
        else:
            return redirect(url_for('login'))
        
    return wrapper

@app.route('/')
def index():
    hotel_name=[]
    hotel_url=[]
    hotel_id = []
    j=0
    with open('city_hotelname_hid_hurl','r',encoding='utf-8') as f:
        for line in f:
            a = line.split()
            
            hotel_name.append(''.join(a[1:len(a)-2]))
            hotel_url.append(a[len(a)-1])
            hotel_id.append(a[len(a)-2])
            if j==10:
                break
            else:
                j+=1
    return render_template('index.html',hotel_name=hotel_name,hotel_url=hotel_url)

@app.route('/login/',methods=['GET','POST'])
def login():
    if request.method == 'GET':
        return render_template('login.html')
    else:
        account = request.form.get('account')
        password = request.form.get('password')
        user_collection=mongo.db.user
        user = user_collection.find_one({"$and":[ {'account':account}
                                             ,{'password':password}
                                             ]})#查無回傳None
        if user:
            session['account'] = user['account']
            session.permanent = True #如果想在31天都不需要登入
            
            return redirect(url_for('index'))
        else:
            return '名稱或密碼錯誤'

@app.route("/logout/", methods=['GET','POST'])
def logout():
    session['account'] = None
    return redirect('/')
@app.route('/register/',methods=['GET','POST'])
def register():
    if request.method == 'GET':
        return render_template('register.html')
    
    
    else:
        account = request.form.get('account')
        password1 = request.form.get('password1')
        password2 = request.form.get('password2')
        
        user_collection=mongo.db.user
        uid = user_collection.estimated_document_count()+1
        if user_collection.find_one({'account':account}):#查無回傳None
            return '該名稱已被註冊，請更換'
        else:
            if password1!=password2:
                return '兩次密碼不相同，請核對後再填寫'
            else:
                user_collection.insert_one({'account':account,\
                            'uid':uid,\
                            'password':password1,\
                            'public_key':'',\
                            'private_key':'',\
                            'ratings':[],\
                            'rcmhotel':[],\
                            'eth':100
                        })
                return redirect(url_for('login'))
        
# @app.route('/login/<id>')
# def login0(id):
#     return render_template('login.html',id=id)

@app.route('/addhotel/',methods=['GET','POST'])
@login_required
def addhotel():
    if request.method =='GET':
        return render_template('addhotel.html')
    else:
        hotelname = request.form.get('hotelname')
        address = request.form.get('address')
        introduction = request.form.get('introduction')
        price = request.form.get('price')
        filepath = request.form.get('filepath')
        
#         basepath = os.path.dirname(__file__)  # 當前文件所再路徑
#         upload_path = os.path.join(basepath, 'static\uploads',secure_filename(f.filename))  
#         #注意：没有的文件夹一定要先创建，不然会提示没有该路径
#         filepath.save(upload_path)
        landlord = session.get('account')#房東
        
        rent_hotel_collection=mongo.db.rent_hotel
        rent_hotel_collection.insert_one({'landlord':landlord,\
                            'hotelname':hotelname,\
                            'address':address,\
                            'introduction':introduction,\
                            'price':price,\
                            'ratings':[]
                        })
        return redirect(url_for('renthotel'))
    

##################
#################
@app.route('/profile/')
def profile():
    _collection=mongo.db.rent_hotel
    context = {
        'rent_hotels':[i for i in user_collection.find({'landlord':session.get('account')}).sort('_id',-1)]#find()return list item
    }
    return render_template('profile.html',**context)

@app.route('/renthotel/')
def renthotel():
    rent_hotel_collection=mongo.db.rent_hotel
    context = {
        'rent_hotels':[i for i in rent_hotel_collection.find({}).sort('_id',-1)]#find()return list item
    }
    return render_template('newhotel.html',**context)
###############
###############
@app.route('/sendmoney/<renthotel_id>/',methods=['GET','POST'])
# @login_required
def sendmoney(renthotel_id):
    if request.method =='GET':
        rent_hotel_collection=mongo.db.rent_hotel
        
        print(renthotel_id)
        return render_template('sendmoney.html',)
    else:
        return redirect(url_for('index'))
    
##########################
##########################
@app.route('/recommend')
@login_required
def recommend():
    hotel_name=[]
    hotel_url=[]
#     hotel_id = []
    account = session.get('account')
    rcmhotel_collection=mongo.db.user
    rcmhotel = rcmhotel_collection.find_one({'account':session.get('account')})
 
    for i in rcmhotel['rcmhotel']:
        hotel_name.append(list(i.values())[0])
    return render_template('index.html',hotel_name=hotel_name,hotel_url=hotel_url)


if __name__=='__main__':    
    app.run(debug=True,use_reloader=False) 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Oct/2019 16:33:30] "GET /profile/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:33:32] "GET /profile/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:33:33] "GET /renthotel/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:33:35] "GET /renthotel/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:33:37] "GET /login/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:33:41] "POST /login/ HTTP/1.1" 302 -
127.0.0.1 - - [15/Oct/2019 16:33:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:33:43] "GET /profile/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:33:44] "GET /renthotel/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:33:45] "GET /recommend HTTP/1.1" 200 -


session get account:  s1051649


127.0.0.1 - - [15/Oct/2019 16:33:47] "GET /recommend HTTP/1.1" 200 -


session get account:  s1051649


127.0.0.1 - - [15/Oct/2019 16:33:48] "GET /renthotel/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:33:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:33:50] "GET /renthotel/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:33:52] "GET /profile/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:34:02] "GET /profile/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:34:04] "GET /renthotel/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:39:46] "GET /profile/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:47:10] "GET /profile/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:47:14] "GET /profile/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:51:10] "GET /profile/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:51:30] "GET /profile/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:52:29] "GET /profile/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:52:44] "GET /profile/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:52:47] "GET /profile/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 16:53:35] "GET /profile

In [47]:
from flask import Flask
from flask_pymongo import PyMongo

app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:27017/recommend"
mongo = PyMongo(app)
hotelname = []
hotelurl = []
rcmhotel_collection=mongo.db.user
rcmhotel = rcmhotel_collection.find_one({'account':'charooooonc'})
 
for i in rcmhotel['rcmhotel']:
    hotelname.append(list(i.values())[0])
    print(list(i.values())[0])

print(hotelname)


異宿風巢景觀旅店 台灣安平
長成民宿 台灣澎湖
花蓮莊家堡田園民宿 台灣吉安
綠原度假旅社 台灣雲林
逸心園民宿 台灣壽豐
旅人船說背包聚落 台灣中西區
旅捷商務旅館 台灣苓雅
Z-house市政府捷運站套房 台灣信義區
雅思德商務公寓 台灣中和
奇豆民宿 台灣花蓮市
['異宿風巢景觀旅店 台灣安平', '長成民宿 台灣澎湖', '花蓮莊家堡田園民宿 台灣吉安', '綠原度假旅社 台灣雲林', '逸心園民宿 台灣壽豐', '旅人船說背包聚落 台灣中西區', '旅捷商務旅館 台灣苓雅', 'Z-house市政府捷運站套房 台灣信義區', '雅思德商務公寓 台灣中和', '奇豆民宿 台灣花蓮市']


In [58]:
from flask import Flask
from flask_pymongo import PyMongo

app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:27017/recommend"
mongo = PyMongo(app)
user_collection=mongo.db.rent_hotel
context = {
        'rent_hotels':[i for i in user_collection.find({'landlord':'s1051649'}).sort('_id',-1)]
}
for i in context.items():
    print(i)

('rent_hotels', [{'_id': ObjectId('5da3b042e754eaddb7e0670f'), 'landlord': 's1051649', 'hotelname': 'hotel1', 'address': 'here', 'introduction': 'you', 'ratings': []}])


In [70]:
from flask import Flask
from flask_pymongo import PyMongo

app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:27017/recommend"
mongo = PyMongo(app)
hotel_id_collection = mongo.db.hotel_id
hotel_id = hotel_id_collection.find({})
for i in hotel_id:
    print(i.get('hotelname'),\
          i.get('landlord'),\
          i.get('location'))#get取代['key']回傳none
    

新驛旅店-復興北路店 None 台灣中山區
優美飯店 None 台灣中山區
圓山大飯店 None 台灣中山區
老爺會館台北林森 None 台灣中山區
華國大飯店 None 台灣中山區
丰居旅店雙連館 None 台灣中山區
方舟旅店-長安復興 None 台灣中山區
台北澄舍商旅 None 台灣中山區
喜瑞飯店 None 台灣中山區
雀客旅館 None 台灣中山區


In [86]:
import os
basepath = os.path.dirname(__file__) 
print(basepath)

NameError: name '__file__' is not defined